<a href="https://colab.research.google.com/github/mikejin01/POSH-Event-Data-Analyzer/blob/main/Posh_event_commission_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#Developed by Mike Jin - 2025
#This scripts help you to upload your Posh Event CSV File
#and Calculate the Commission by Codes
from google.colab import drive
import pandas as pd
from IPython.display import display

# Mount Google Drive
drive.mount('/content/drive')

# Now you can browse in the left sidebar (Files tab) to copy the path
# Paste that path here, e.g.:
file_path = '/content/drive/My Drive/Events/20250823-anime-rave-posh-final.csv'

# Read CSV
df = pd.read_csv(file_path)

# strip spaces from column names just in case
df.columns = df.columns.str.strip()

# Convert columns to numeric safely
df["Order Subtotal"] = pd.to_numeric(
    df["Order Subtotal"].astype(str).str.replace(r"[\$,]", "", regex=True), errors="coerce"
)
df["Tickets Scanned"] = pd.to_numeric(df["Tickets Scanned"], errors="coerce")




#__________________________ For Commission Calculation

# Filter rows: promo code not empty & order subtotal > 0
filtered = df[(df["Promo Code"].notna()) & (df["Promo Code"] != "") & (df["Order Subtotal"] > 0)]

# Group and calculate
promo_totals = (
    filtered.groupby("Promo Code", as_index=False)["Order Subtotal"].sum()
    .sort_values(by="Order Subtotal", ascending=False)
)

# Calculate Commission (numeric first)
promo_totals["Commission"] = promo_totals["Order Subtotal"] * 0.20

# Sort descending to see biggest totals first
promo_totals = promo_totals.sort_values(by="Order Subtotal", ascending=False)


# Format both as $ strings AFTER calculations
promo_totals["Order Subtotal"] = promo_totals["Order Subtotal"].apply(lambda x: "${:,.2f}".format(x))
promo_totals["Commission"] = promo_totals["Commission"].apply(lambda x: "${:,.2f}".format(x))

#promo_totals


#__________________________ For Attendance Rate

# Define paid vs non-paid
df["Paid Status"] = df["Order Subtotal"].apply(lambda x: "Paid" if x > 0 else "Non-Paid")

# Define attended (scanned > 0)
df["Attended"] = df["Tickets Scanned"] > 0

# Group by Paid Status
attendance_summary = (
    df.groupby("Paid Status")
      .agg(
          Total_Orders=("Order Number", "count"),
          Attended_Orders=("Attended", "sum")
      )
)

attendance_summary["Attendance Rate"] = (
    (attendance_summary["Attended_Orders"] / attendance_summary["Total_Orders"]) * 100
).round(2).astype(str) + '%'

# Show formatted table
#attendance_summary.reset_index()


display(promo_totals)
display(attendance_summary.reset_index())




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Promo Code,Order Subtotal,Commission
5,sojuking,$395.00,$79.00
0,4amwav,$175.00,$35.00
3,mixee,$105.00,$21.00
1,anime,$25.00,$5.00
2,estella,$25.00,$5.00
4,remix,$10.00,$2.00


,Paid Status,Total_Orders,Attended_Orders,Attendance Rate
0,Non-Paid,97,26,26.8%
1,Paid,83,63,75.9%
